In [1]:
import pandas as pd
pd.set_option("display.max_columns", 25)

# Load the Excel file
file_path = './Datasets/Personality Survey Data.xlsx'

# Load the 'Personal Characteristics Survey' sheet into a pandas DataFrame
survey_data = pd.read_excel(file_path, sheet_name='Personal Characteristics Survey')

# Remove the first few rows to clean up the headers
cleaned_survey_data = pd.read_excel(file_path, sheet_name='Personal Characteristics Survey', header=1)


# Drop any rows where all columns are NaN
cleaned_survey_data.dropna(how='all', inplace=True)

# Reset index after dropping NaN rows
cleaned_survey_data.reset_index(drop=True, inplace=True)

# Remove columns where all values are NaN
cleaned_survey_data = cleaned_survey_data.dropna(axis=1, how='all')



# Reassign the first row to be the column headers
cleaned_survey_data.columns = cleaned_survey_data.iloc[0]

# Drop the first row now that it's set as column headers
cleaned_survey_data = cleaned_survey_data.drop(0).reset_index(drop=True)


retailer_index = cleaned_survey_data[cleaned_survey_data['PLAYER NAME'] == 'retailer22'].index.min()

# If the index is found, delete all rows after the first instance of "retailer22"
if pd.notna(retailer_index):
    cleaned_survey_data = cleaned_survey_data.iloc[:retailer_index + 1]

columns_to_drop = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10']

# Drop those columns if they exist in the dataframe
cleaned_survey_data = cleaned_survey_data.drop(columns=columns_to_drop, errors='ignore')

cleaned_survey_data = cleaned_survey_data.rename(columns={cleaned_survey_data.columns[3]: 'Self Esteem Average'})

value_mapping = {
    'Completely Disagree': 1,
    'Somewhat Disagree': 2,
    'Moderately Disagree': 3,
    'Not Sure': 4,
    'Somewhat Agree': 5,
    'Moderately Agree': 6,
    'Completely Agree': 7
}

# Replace the values in columns R1 to R5 using the mapping
columns_to_replace = ['R1', 'R2', 'R3', 'R4', 'R5']
cleaned_survey_data[columns_to_replace] = cleaned_survey_data[columns_to_replace].replace(value_mapping)


cleaned_survey_data["Regret Scale Average"] = (cleaned_survey_data['R1'].astype(int) + cleaned_survey_data['R2'].astype(int) + cleaned_survey_data['R3'].astype(int) + cleaned_survey_data['R4'].astype(int) + cleaned_survey_data['R5'].astype(int)) / 5


cleaned_survey_data = cleaned_survey_data.dropna(axis=1, how='all')

cleaned_survey_data = cleaned_survey_data.drop(columns=['R1', 'R2', 'R3', 'R4', 'R5'], errors='ignore')

# Move "Regret Scale Average" next to "Self Esteem Average"
# First, remove the column "Regret Scale Average" and store it temporarily
regret_scale_average = cleaned_survey_data.pop('Regret Scale Average')

# Insert the "Regret Scale Average" column right after "Self Esteem Average"
self_esteem_index = cleaned_survey_data.columns.get_loc('Self Esteem Average')
cleaned_survey_data.insert(self_esteem_index + 1, 'Regret Scale Average', regret_scale_average)

rl_value_mapping = {
    'RL1': 2,
    'RL2': 1.5,
    'RL3': 1,
    'RL4': 0.5,
    'RL5': 0,
    'RL6': -0.5,
    'RL7': -1,
    'RL8': -1.5
}

# Initialize a new column "Tossing Coin Risk Averse Coefficient"
cleaned_survey_data['Tossing Coin Risk Averse Coefficient'] = None

# Assign values based on the first occurrence of RL value equal to 0

for index, row in cleaned_survey_data.iterrows():
    for rl_column in ['RL 1', 'RL 2', 'RL 3', 'RL 4', 'RL 5', 'RL 6', 'RL 7']:
        if rl_column == "RL 7":
            if row[rl_column] == 0:  # Check if the RL value is 0
                cleaned_survey_data.loc[index, 'Tossing Coin Risk Averse Coefficient'] = rl_value_mapping.get(rl_column.replace(' ', ''))
                break
            else:
                cleaned_survey_data.loc[index, 'Tossing Coin Risk Averse Coefficient'] = rl_value_mapping.get('RL8'.replace(' ', ''))
                break

        if row[rl_column] == 0:  # Check if the RL value is 0
            cleaned_survey_data.loc[index, 'Tossing Coin Risk Averse Coefficient'] = rl_value_mapping.get(rl_column.replace(' ', ''))
            break
    

def calculate_risk_value(row):
    # Calculate Risk1 value
    risk1_value = 1 if row['Risk1'] < 100 else 0 if row['Risk1'] == 100 else -1

    # Calculate Risk 2 value
    risk2_value = 0.5 if row['Risk 2'] == '100 TL' else -0.5 if row['Risk 2'] == 'Ticket' else 0

    # Calculate Risk 3 value
    risk3_value = 1 if row['Risk 3'] < 100 else 0 if row['Risk 3'] == 100 else -1

    # Sum Risk values
    total_risk = risk1_value + risk2_value + risk3_value
    return total_risk

# Apply the risk calculation
cleaned_survey_data['Risk Sum'] = cleaned_survey_data.apply(calculate_risk_value, axis=1)

# Calculate the "Risk Averse Coefficient" by summing the Risk Sum and Tossing Coin Risk Averse Coefficient
cleaned_survey_data['Risk Averse Coefficient'] = cleaned_survey_data['Risk Sum'] + pd.to_numeric(cleaned_survey_data['Tossing Coin Risk Averse Coefficient'], errors='coerce')

# Drop the unnecessary columns: Risk1, Risk 2, Risk 3, RL1 through RL7, and Tossing Coin Risk Averse Coefficient
columns_to_drop = ['Risk1', 'Risk 2', 'Risk 3', 'RL 1', 'RL 2', 'RL 3', 'RL 4', 'RL 5', 'RL 6', 'RL 7', 'Tossing Coin Risk Averse Coefficient', 'Risk Sum']
cleaned_survey_data = cleaned_survey_data.drop(columns=columns_to_drop, errors='ignore')

risk_averse_coefficient = cleaned_survey_data.pop('Risk Averse Coefficient')

# Insert the "Risk Averse Coefficient" column right after "Regret Scale Average"
regret_scale_index = cleaned_survey_data.columns.get_loc('Regret Scale Average')
cleaned_survey_data.insert(regret_scale_index + 1, 'Risk Averse Coefficient', risk_averse_coefficient)

points = {
    'F3': -2,  # 10 TL offer acceptance
    'F4': -1,  # 20 TL offer acceptance
    'F5': 0,  # 30 TL offer acceptance
    'F6': 1,  # 40 TL offer acceptance
    'F7': 2  # 50 TL offer acceptance
}

# Calculate Fairness for Q1 and Q2 based on F6 and F7
cleaned_survey_data['Fairness_Q1'] = (100 - cleaned_survey_data['F1']) / 100
cleaned_survey_data['Fairness_Q2'] = (100 - cleaned_survey_data['F2']) / 100

# Calculate Fairness_Q3 based on weighted points from F1 to F5
cleaned_survey_data['Fairness_Q3'] = cleaned_survey_data[[col for col in points.keys()]].multiply(points.values(), axis=1).sum(axis=1) / 10

# Calculate the overall Fairness Index
cleaned_survey_data['Fairness Index'] = (cleaned_survey_data['Fairness_Q1'] + cleaned_survey_data['Fairness_Q2'] + cleaned_survey_data['Fairness_Q3']) / 3

# Drop the F1 to F7 columns
cleaned_survey_data = cleaned_survey_data.drop(columns=['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7'], errors='ignore')

cleaned_survey_data = cleaned_survey_data.drop(["Fairness_Q1", "Fairness_Q2", "Fairness_Q3"], axis=1)

cleaned_survey_data = cleaned_survey_data.loc[:, cleaned_survey_data.columns.notna()]

# Display the cleaned data
cleaned_survey_data.head()


cleaned_survey_data.to_csv('adjusted_reponse_survey.csv', index=False)

C:\Users\telat\AppData\Local\Temp\ipykernel_73592\3665882903.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_survey_data[columns_to_replace] = cleaned_survey_data[columns_to_replace].replace(value_mapping)


In [2]:
cleaned_survey_data.head(20)

,PLAYER NAME,Gender,Date of birth,Self Esteem Average,Regret Scale Average,Risk Averse Coefficient,Fairness Index
0,manufacturer1,Male,1993-10-11,1.9,3.6,-0.5,0.433333
1,manufacturer2,Female,1990-06-03,2.9,4.8,1.5,0.366667
2,manufacturer3,Male,1991-07-05,1.7,5.8,3.5,0.433333
3,manufacturer4,Female,1991-08-08,2.0,4.2,1.0,0.4
4,manufacturer5,Male,1988-05-18,2.6,5.4,3.0,0.003333
5,manufacturer6,Male,1990-09-09,1.2,5.4,0.5,0.3
6,manufacturer7,Female,1992-05-10,1.9,4.0,2.0,0.166667
7,manufacturer8,Male,1992-04-10,2.6,5.2,-1.5,0.4
8,manufacturer9,Male,1990-09-11,2.8,3.4,2.5,0.283333
9,manufacturer10,Female,1990-06-07,3.0,3.4,2.5,0.3


In [3]:
cleaned_survey_data


,PLAYER NAME,Gender,Date of birth,Self Esteem Average,Regret Scale Average,Risk Averse Coefficient,Fairness Index
0,manufacturer1,Male,1993-10-11,1.9,3.6,-0.5,0.433333
1,manufacturer2,Female,1990-06-03,2.9,4.8,1.5,0.366667
2,manufacturer3,Male,1991-07-05,1.7,5.8,3.5,0.433333
3,manufacturer4,Female,1991-08-08,2.0,4.2,1.0,0.4
4,manufacturer5,Male,1988-05-18,2.6,5.4,3.0,0.003333
5,manufacturer6,Male,1990-09-09,1.2,5.4,0.5,0.3
6,manufacturer7,Female,1992-05-10,1.9,4.0,2.0,0.166667
7,manufacturer8,Male,1992-04-10,2.6,5.2,-1.5,0.4
8,manufacturer9,Male,1990-09-11,2.8,3.4,2.5,0.283333
9,manufacturer10,Female,1990-06-07,3.0,3.4,2.5,0.3


In [8]:
import pandas as pd

# Assume 'df' is your concatenated DataFrame
# For example:
# df = pd.read_csv('your_concatenated_dataframe.csv')

# Create 'Role' column based on 'PLAYER NAME'
cleaned_survey_data['Role'] = cleaned_survey_data['PLAYER NAME'].apply(lambda x: 'Manufacturer' if 'manufacturer' in x.lower() else 'Retailer')

# Separate manufacturers and retailers
manufacturers = cleaned_survey_data[cleaned_survey_data['Role'] == 'Manufacturer'].reset_index(drop=True)
retailers = cleaned_survey_data[cleaned_survey_data['Role'] == 'Retailer'].reset_index(drop=True)

# Rename columns to distinguish between manufacturer and retailer
manufacturers.columns = ['Manufacturer_' + col if col not in ['PLAYER NAME', 'Role'] else col for col in manufacturers.columns]
retailers.columns = ['Retailer_' + col if col not in ['PLAYER NAME', 'Role'] else col for col in retailers.columns]

# Merge the dataframes side by side
combined = pd.concat([manufacturers.reset_index(drop=True), retailers.reset_index(drop=True)], axis=1)

# Compute combined metrics
combined['Avg_SelfEsteem'] = (combined['Manufacturer_Self Esteem Average'] + combined['Retailer_Self Esteem Average']) / 2
combined['Diff_SelfEsteem'] = combined['Manufacturer_Self Esteem Average'] - combined['Retailer_Self Esteem Average']

combined['Avg_RegretScale'] = (combined['Manufacturer_Regret Scale Average'] + combined['Retailer_Regret Scale Average']) / 2
combined['Diff_RegretScale'] = combined['Manufacturer_Regret Scale Average'] - combined['Retailer_Regret Scale Average']

combined['Avg_RiskAverseCoeff'] = (combined['Manufacturer_Risk Averse Coefficient'] + combined['Retailer_Risk Averse Coefficient']) / 2
combined['Diff_RiskAverseCoeff'] = combined['Manufacturer_Risk Averse Coefficient'] - combined['Retailer_Risk Averse Coefficient']
combined['RiskAverse_Interaction'] = combined['Manufacturer_Risk Averse Coefficient'] * combined['Retailer_Risk Averse Coefficient']

combined['Avg_FairnessIndex'] = (combined['Manufacturer_Fairness Index'] + combined['Retailer_Fairness Index']) / 2
combined['Diff_FairnessIndex'] = combined['Manufacturer_Fairness Index'] - combined['Retailer_Fairness Index']
combined['Fairness_Interaction'] = combined['Manufacturer_Fairness Index'] * combined['Retailer_Fairness Index']

# Conserve gender and date of birth columns
# (Columns are already included with distinct prefixes)

# Handle missing values
combined.fillna(0, inplace=True)


combined.to_csv('adjusted_reponse_survey.csv', index=False)

# Display the combined DataFrame
combined.head()

,PLAYER NAME,Manufacturer_Gender,Manufacturer_Date of birth,Manufacturer_Self Esteem Average,Manufacturer_Regret Scale Average,Manufacturer_Risk Averse Coefficient,Manufacturer_Fairness Index,Role,PLAYER NAME,Retailer_Gender,Retailer_Date of birth,Retailer_Self Esteem Average,...,Retailer_Fairness Index,Role,Avg_SelfEsteem,Diff_SelfEsteem,Avg_RegretScale,Diff_RegretScale,Avg_RiskAverseCoeff,Diff_RiskAverseCoeff,RiskAverse_Interaction,Avg_FairnessIndex,Diff_FairnessIndex,Fairness_Interaction
0,manufacturer1,Male,1993-10-11,1.9,3.6,-0.5,0.433333,Manufacturer,retailer1,Male,1987-03-16,2.8,...,0.333333,Retailer,2.35,-0.9,4.5,-1.8,1.00,-3.0,-1.25,0.383333,0.1,0.144444
1,manufacturer2,Female,1990-06-03,2.9,4.8,1.5,0.366667,Manufacturer,retailer2,Female,1992-02-03,2.6,...,0.266667,Retailer,2.75,0.3,4.9,-0.2,2.25,-1.5,4.50,0.316667,0.1,0.097778
2,manufacturer3,Male,1991-07-05,1.7,5.8,3.5,0.433333,Manufacturer,retailer3,Male,1990-11-02,1.4,...,0.233333,Retailer,1.55,0.3,5.0,1.6,1.25,4.5,-3.50,0.333333,0.2,0.101111
3,manufacturer4,Female,1991-08-08,2.0,4.2,1.0,0.4,Manufacturer,retailer4,Male,1992-02-10,2.7,...,0.403333,Retailer,2.35,-0.7,3.7,1.0,1.00,0.0,1.00,0.401667,-0.003333,0.161333
4,manufacturer5,Male,1988-05-18,2.6,5.4,3.0,0.003333,Manufacturer,retailer5,Male,1990-06-30,2.5,...,0.2,Retailer,2.55,0.1,4.9,1.0,2.25,1.5,4.50,0.101667,-0.196667,0.000667
